In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import contextlib
import os

import rubin_sim.maf as maf

from u_band_strat import data_dir, fig_dir, interpolate_number_density, colors

In [3]:
# Set min SNR in dropout band, and dropout threshold
snr_min = 3
dropout = 1

# Set the fraction of WFD used
f_wfd = 0.75

In [ ]:
metrics = dict()
for year in [1, 10]:
    metrics[year] = dict()
    for expt in [30, 38, 45, 60]:
        metrics[year][expt] = dict()
        for scale in [1.0, 1.1, 1.2, 1.5]:
            if expt == 60 and scale == 1.5:
                continue

            # Load the dropout band map
            scale_str = str(scale).replace(".", "_")
            with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
                u5 = maf.MetricBundle.load(
                    data_dir
                    / "m5_maps"
                    / f"internal_u_expt{expt}_nscale{scale_str}v3_4_{year}yrs_ExgalM5_u.npz",
                ).metric_values

            # Determine the cut
            cut = (
                np.quantile(u5[~u5.mask], 1 - f_wfd)
                - dropout
                + 2.5 * np.log10(5 / snr_min)
            )

            density = interpolate_number_density("u", cut, cut)
            metrics[year][expt][scale] = np.ma.mean(density)

metrics = {year: pd.DataFrame(metrics[year]) for year in metrics}

In [ ]:
# Create the figure
fig, axes = plt.subplots(1, 2, figsize=(6.4, 3.2), constrained_layout=True, dpi=150)

for i, year in enumerate([1, 10]):
    # Extract metric values
    table = metrics[year]
    vals = table.to_numpy().T

    # Print absolute density
    print(f"Density for year {year} is {vals[0, 0]:.0f}")

    # Normalize metric to baseline strategy
    vals /= vals[0, 0]

    # Plot metric colors
    axes[i].imshow(
        vals,
        cmap="coolwarm",
        vmin=np.nanmin(vals),
        vmax=np.nanmax(vals),
        origin="lower",
    )

    # Plot metric labels
    for (j, k), z in np.ndenumerate(vals):
        if np.isfinite(z):
            axes[i].text(k, j, f"{z:.2f}", ha="center", va="center", fontsize=12)

    # Set axes properties
    axes[i].set(
        xticks=np.arange(4),
        xticklabels=table.index,
        xlabel="Fraction of $u$ band visits",
        yticks=np.arange(4),
        yticklabels=table.columns,
        ylabel="$u$ band exposure time",
        title=f"Year {year}",
    )

    # Box baseline v3.4
    axes[i].plot(
        [-0.49, 0.5, 0.5, -0.49, -0.49],
        [-0.48, -0.48, 0.5, 0.5, -0.48],
        c="C3",
        lw=1.2,
        ls="-",
        zorder=10,
    )

    # Box the fiducial strategy
    axes[i].plot(
        [0.5, 1.5, 1.5, 0.5, 0.5],
        [0.5, 0.5, 1.5, 1.5, 0.5],
        c="C3",
        lw=1.2,
        ls="--",
    )

# Set the title
fig.suptitle("Relative number density of $u$-band dropouts")

fig.savefig(fig_dir / "number_density_grid.pdf")

In [ ]:
strategies = [
    "baseline_v3_4_{year}yrs_ExgalM5_{band}.npz",
    "internal_u_expt38_nscale1_1v3_4_{year}yrs_ExgalM5_{band}.npz",
    "baseline_v3_5_{year}yrs_ExgalM5_{band}.npz",
    "too_v3_5_{year}yrs_ExgalM5_{band}.npz",
    "baseline_v3_6_{year}yrs_ExgalM5_{band}.npz",
    "baseline_v4_0_{year}yrs_ExgalM5_{band}.npz",
]

det_bands = dict(u="r", g="i", r="z", i="z", z="y")

evolution = dict()
for year in [1, 10]:
    evolution[year] = dict()
    for band in "ugr":
        evolution[year][band] = []
        for strat in strategies:
            # Get the detection band
            det_band = det_bands[band]

            # Load maps for dropout and detection bands
            with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
                drop = maf.MetricBundle.load(
                    data_dir / "m5_maps" / strat.format(year=year, band=band),
                ).metric_values
                det = maf.MetricBundle.load(
                    data_dir / "m5_maps" / strat.format(year=year, band=det_band),
                ).metric_values

            # Determine the cut
            cut = min(
                np.quantile(drop[~drop.mask], 1 - f_wfd)
                - dropout
                + 2.5 * np.log10(5 / snr_min),
                np.quantile(det[~det.mask], 1 - f_wfd),
            )

            density = interpolate_number_density(band, cut, cut)
            evolution[year][band].append(np.ma.mean(density))

evolution = {year: pd.DataFrame(evolution[year]) for year in evolution}

In [ ]:
fig, axes = plt.subplots(2, 1, dpi=150, figsize=(5, 4.5))

for i, year in enumerate(evolution):
    # Plot evolution of each dropout sample
    for band in evolution[year]:
        density = evolution[year][band]
        axes[i].plot(density / density[0], c=colors[band], label=f"${band}$ dropouts")

    # Line at 1 to guide the eye
    axes[i].axhline(1, c="gray", ls="--")

    axes[i].set(
        title=f"Year {year}",
        xticks=np.arange(6),
        xticklabels=[
            "v3.4",
            "v3.4\n[1.1x, 38s $u$]",
            "v3.5",
            "v3.5\nw/ ToOs",
            "v3.6",
            "v4.0",
        ],
        ylim=(0, 2),
        ylabel="Relative number density",
    )

    axes[i].legend(handlelength=1, fontsize=8, frameon=False, borderpad=0)

fig.subplots_adjust(hspace=0.55)
fig.savefig(fig_dir / "density_evolution.pdf", bbox_inches="tight")